# Experiments in SMTLib Translation #

Let us look at a simple example that poses problems for the Z3/CVC4 backend:

In [1]:
:solve z3 f = {1|->3, 2|->6} & r = f~[{6}]

CommandExecutionException: :solve: Computation not completed: no solution found (but one might exist)

To understand why this simple constraint cannot be solved, we have to know how the translation works:
The relational inverse gets translated into two universal quantifications for SMTLib:
```
 x = y~
<=>
 !(st11,st12).(st11 |-> st12 : x => st12 |-> st11 : y) & 
 !(st11,st12).(st12 |-> st11 : y => st11 |-> st12 : x))
```
Similarly, r = f[s] is translated as follows:
```
 r = f[s]
<=>
 !st27.(st27 : r => #st26.(st26 |-> st27 : f & st26 : s) & 
 !st27.(#st26.(st26 |-> st27 : f & st26 : s) => st27 : r)
```
The resulting predicate (without the inverse and image operators) is the following, which Z3 cannot solve (but ProB can).

In [2]:
:prettyprint f = {(1|->3),(2|->6)} &
#st13.(r = st13 & (
    !st15.(st15 : st13 => #st14.(#st16.(st14 |-> st15 : st16 & 
    (!(st17,st18).(st17 |-> st18 : st16 => st18 |-> st17 : f) & 
     !(st17,st18).(st18 |-> st17 : f => st17 |-> st18 : st16))) & st14 : {6})) & 
     !st15.(#st14.(#st19.(st14 |-> st15 : st19 & (!(st20,st21).(st20 |-> st21 : st19 => st21 |-> st20 : f) &
     !(st20,st21).(st21 |-> st20 : f => st20 |-> st21 : st19))) & st14 : {6}) => st15 : st13)))

f = {(1↦3),(2↦6)} ∧ (∃ /* LET */ (st13).( (st13)=r ∧ ∀st15Â·(st15 ∈ st13 ⇒ ∃st16Â·(6 ↦ st15 ∈ st16 ∧ (∀(st17,st18)Â·(st17 ↦ st18 ∈ st16 ⇒ st18 ↦ st17 ∈ f) ∧ ∀(st17,st18)Â·(st18 ↦ st17 ∈ f ⇒ st17 ↦ st18 ∈ st16)))) ∧ ∀st15Â·(∃st19Â·(6 ↦ st15 ∈ st19 ∧ (∀(st20,st21)Â·(st20 ↦ st21 ∈ st19 ⇒ st21 ↦ st20 ∈ f) ∧ ∀(st20,st21)Â·(st21 ↦ st20 ∈ f ⇒ st20 ↦ st21 ∈ st19))) ⇒ st15 ∈ st13)))

In [3]:
:time :solve prob f = {(1|->3),(2|->6)} &
#st13.(r = st13 & (
    !st15.(st15 : st13 => #st14.(#st16.(st14 |-> st15 : st16 & 
    (!(st17,st18).(st17 |-> st18 : st16 => st18 |-> st17 : f) & 
     !(st17,st18).(st18 |-> st17 : f => st17 |-> st18 : st16))) & st14 : {6})) & 
     !st15.(#st14.(#st19.(st14 |-> st15 : st19 & (!(st20,st21).(st20 |-> st21 : st19 => st21 |-> st20 : f) &
     !(st20,st21).(st21 |-> st20 : f => st20 |-> st21 : st19))) & st14 : {6}) => st15 : st13)))



Execution time: 0.194111316 seconds

TRUE

Solution:
	r = {2}
	f = {(1↦3),(2↦6)}

Experiment in Rodin: adding r={2} as theorem and use SMT Plugin using the technique suggested by Laurent:

pour voir la traduction d’Event-B vers SMT, créer un fichier texte contenant
```
 org.eventb.smt.core/debug = true
 org.eventb.smt.core/debug/translator = true
 org.eventb.smt.core/debug/translator_details = true
```
et lancer Rodin avec les paramètres `-debug <text-file>`.

À chaque fois qu’un solveur SMT est lancé, le contenu du fichier SMT sera affiché dans la console.

```
; translated from Event-B with the PP approach of Rodin SMT Plugin

(set-info :status unsat)
(set-logic AUFLIA)
(declare-fun f (Int Int) Bool)
(declare-fun r (Int) Bool)

(assert (and 
            (forall ((x Int) (x0 Int)) 
                (= 
                    (f x x0) 
                    (or 
                        (and 
                            (= x 1) 
                            (= x0 3)) 
                        (and 
                            (= x 2) 
                            (= x0 6))))) 
            (forall ((x1 Int)) 
                (= 
                    (r x1) 
                    (exists ((x2 Int)) 
                        (and 
                            (= x2 6) 
                            (f x1 x2)))))))
(assert (not 
            (forall ((x3 Int)) 
                (= 
                    (r x3) 
                    (= x3 2)))))
(check-sat)
```